In [331]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
sys.path.append('..')
from gobancv.roi import find_roi
%reload_ext autoreload
%autoreload 2


def imshow(img, size=(5, 5)):
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    plt.rcParams['figure.figsize'] = size
    plt.imshow(img)
    plt.yticks([])
    plt.xticks([])
    plt.show()

IMAGE_PATH = "../data/static/"

imgs = [cv.imread(IMAGE_PATH + img) for img in os.listdir(IMAGE_PATH) if img.split('.')[-1] in ['jpg', 'jpeg', 'png']]

In [332]:
from sklearn.cluster import DBSCAN

def dedup(lines):
    rhs = lines[:, :, 1]
    directions = DBSCAN(eps=0.2, min_samples=2).fit(rhs)

    cur = []
    for direction in np.unique(directions.labels_):
        approx_dir = np.mean(rhs[directions.labels_ == direction])
        dirlines = lines[directions.labels_ == direction][:, :, 0]
        clustering = DBSCAN(eps=5, min_samples=1).fit(dirlines)
        for cluster in np.unique(clustering.labels_):
            cur.append((np.mean(dirlines[clustering.labels_ == cluster], axis=0).item(), approx_dir))
    cur = np.array(cur).reshape(-1, 1, 2)
    print(cur.shape)
    return cur



In [333]:
from collections import Counter
from gobancv.grid import get_intersections, draw_intersections


In [1]:
from gobancv.grid import draw_lines
from gobancv.grid import get_intersection

def segment_by_angle(lines):
    """Segment lines using DBSCAN by their angle, return 2 largest clusters"""
    rhs = lines[:, :, 1]
    clusters = DBSCAN(eps=0.2, min_samples=3).fit(rhs)
    c = Counter(filter(lambda a: a != -1, list(clusters.labels_)))
    largest = c.most_common(2)
    a = lines[clusters.labels_ == largest[0][0]]
    b = lines[clusters.labels_ == largest[1][0]]
    return a, b 

def find_corners(a, b):
    lowest_a = a[np.argmin(a[:, :, 0])]
    highest_a = a[np.argmax(a[:, :, 0])]
    lowest_b = b[np.argmin(b[:, :, 0])]
    highest_b = b[np.argmax(b[:, :, 0])]
    return [
        get_intersection(lowest_a, lowest_b),
        get_intersection(lowest_a, highest_b),
        get_intersection(highest_a, highest_b),
        get_intersection(highest_a, lowest_b),
    ]

def find_lines(img, mask, debug=False):
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    filtered = cv.bilateralFilter(gray, 3, 75, 75)
    adaptive = cv.adaptiveThreshold(
        filtered, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV,
        blockSize=15, # to tune
        C=8)          # to tune
    print(adaptive.shape, mask.shape)
    adaptive = cv.bitwise_and(adaptive, mask)

    # repeat until we get good amount of lines
    param=200
    lines = cv.HoughLines(adaptive, 1, np.pi/180, param)
    print(len(lines))
    while len(lines) < 18:
        adaptive = cv.dilate(adaptive, np.ones((3, 3), np.uint8))
        print(param)
        lines = cv.HoughLines(adaptive, 1, np.pi/180, param)

    # debug
    if debug:
        imgc = img.copy()
        draw_lines(imgc, lines)
        adaptive = cv.cvtColor(adaptive, cv.COLOR_GRAY2BGR)
        filtered = cv.cvtColor(filtered, cv.COLOR_GRAY2BGR)
        imshow(np.concatenate([filtered, adaptive, imgc], axis=1), size=(10, 5))
    return lines

def get_warp_matrix(img):
    copied = img.copy()
    hull = find_roi(img, False)
    mask = np.zeros(img.shape[:2], np.uint8)
    cv.fillConvexPoly(mask, hull, 255)

    lines = find_lines(img, mask, False)
    a, b = segment_by_angle(lines)
    draw_lines(img, a)
    draw_lines(img, b, color=(0, 255, 0))

    corners = find_corners(a, b) 
    draw_intersections(img, corners)

    imshow(img, size=(10, 5))

    w = min(img.shape[0], img.shape[1])
    dest_points = np.array([[0, 0], [0, w], [w, w], [w, 0]], dtype=np.float32)
    M = cv.getPerspectiveTransform(np.array(corners, dtype=np.float32), dest_points)
    warped = cv.warpPerspective(copied, M, (w+10, w+10))
    imshow(warped)


ImportError: cannot import name 'get_intersection' from 'gobancv.grid' (/home/adam/5sem/cv/project_go/gobancv/grid.py)